In [1]:
import pandas as pd
import numpy as np
import os
from string import digits
import datetime as dt
from alpha_vantage.timeseries import TimeSeries
from multiprocessing import Pool, cpu_count

In [2]:
# TODO: Add this to envy
# quandl.ApiConfig.api_key = "s4sS8qy8LcvoW3Ls3dRT"
os.environ["ALPHAVANTAGE_API_KEY"] = "100HED30C7HC9JMI" #alphavantage

In [3]:
def remove_digits(input_str):
    remove_digits = str.maketrans('', '', digits)
    res = input_str.translate(remove_digits)
    return res

In [4]:
class Stock(object):
    '''
    
    '''
    def __init__(self, tickers_list=[],stock_index='SP500',
                 price_type=None, ts=None, after_hours=False):
        self.tickers_df = None
        self.tickers_list = tickers_list
        self.stock_index = stock_index
        self.price_type = price_type
        self.ts = ts
        self.prices_df = None
        self.after_hours = after_hours
        
        
    def get_tickers_index(self):
        '''
        Get the list of all stocks in a given index.
        '''
        if self.stock_index == 'SP500':
            # There are 2 tables on the Wikipedia page, the first contains tickers
            payload = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies', header=0)
            self.tickers_df = payload[0]
            if len(self.tickers_df.index) >= 450:
                self.tickers_df = self.tickers_df.head(2) #TODO: remove this
                self.tickers_list = list(self.tickers_df['Symbol'])
            else:
                ValueError('Check wikipedia data source for SP 500 at \
                             https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
        else:
            e = str('stock_index value '+ self.stock_index + ' not defined')
            raise ValueError(e)
        

    def get_list_stock_prices(self):
        '''
        Fetch stock price for list of tickers.
        '''
        if not self.tickers_list:
            raise KeyError('tickers_list not provided, either pass as argument or \
                call get_tickers_index() method with index name to fetch tickers.')
        self.pool = Pool(cpu_count()-1)
        prices_df_list = self.pool.map_async(self.get_individual_stock_price, self.tickers_list).get()
        self.prices_df = pd.concat(prices_df_list) \
            .reset_index() \
            .rename({'index':'ts'}, axis=1) \
            .set_index(['ticker', 'ts'])
        
    
    def get_individual_stock_price(self, stock_ticker):
        '''
        Fetch stock price using the Alphavantage API. 
        '''
        if self.price_type == 'intraday':
            price, meta_data = self.ts.get_intraday(stock_ticker)
            price_df = pd.DataFrame(price).transpose()
            price_df.index = pd.to_datetime(price_df.index)
            if not self.after_hours:
                price_df = price_df[(price_df.index.time>=dt.time(9,30)) & 
                                    (price_df.index.time<=dt.time(16,0))]
            price_df.columns = [remove_digits(col) for col in price_df.columns]
            price_df.columns = [col.replace('. ', '') for col in price_df.columns]
            price_df['ticker'] = stock_ticker
            return price_df
    
    
    def __getstate__(self):
        self_dict = self.__dict__.copy()
        del self_dict['pool']
        return self_dict

    def __setstate__(self, state):
        self.__dict__.update(state)

### MAIN

In [5]:
ts = TimeSeries()

In [6]:
s = Stock(price_type='intraday', ts=ts, stock_index='SP500')

In [7]:
s.get_tickers_index()
s.tickers_list

['MMM', 'ABT']

In [8]:
s.get_list_stock_prices()

In [9]:
s.prices_df

open      high       low     close  volume
ticker ts                                                                 
MMM    2020-07-31 16:00:00  149.8600  150.6300  149.7700  150.5300  392033
       2020-07-31 15:45:00  149.8800  149.9900  149.6200  149.8400  132858
       2020-07-31 15:30:00  149.5200  150.0000  149.3800  149.8550  108522
       2020-07-31 15:15:00  149.4600  149.7300  149.4201  149.5300   77637
       2020-07-31 15:00:00  149.6200  149.6200  149.2600  149.4400   64927
       2020-07-31 14:45:00  149.1200  149.6500  149.0800  149.6000   55627
       2020-07-31 14:30:00  148.9400  149.1000  148.9300  149.0800   55737
       2020-07-31 14:15:00  149.0600  149.3700  148.9350  148.9500   67152
       2020-07-31 14:00:00  149.1100  149.2800  149.0300  149.1000   95505
       2020-07-31 13:45:00  149.4300  149.5600  149.0941  149.1300   49801
       2020-07-31 13:30:00  149.0500  149.4550  149.0500  149.4550   45143
       2020-07-31 13:15:00  148.9800  149.3300  148.8000  149.0250   63056
       2020-07-31 13:00:00  149.1300  149.2100  148.9300  148.9966   74089
       2020-07-31 12:45:00  149.2100  149.3650  148.9300  149.1100   51259
       2020-07-31 12:30:00  149.0600  149.5400  148.8700  149.2850   73958
       2020-07-31 12:15:00  149.3400  149.4200  149.0300  149.0500   78493
       2020-07-31 12:00:00  149.5500  149.7250  149.3500  149.3500   58678
       2020-07-31 11:45:00  149.9900  150.0400  149.4400  149.4900   75494
       2020-07-31 11:30:00  149.7700  150.1000  149.5901  149.9700   73183
       2020-07-31 11:15:00  149.5900  150.0600  149.2500  149.7500   85519
       2020-07-31 11:00:00  149.8600  150.0700  149.4501  149.6000   68776
       2020-07-31 10:45:00  150.2200  150.2500  149.6430  149.9500   81874
       2020-07-31 10:30:00  150.4700  150.7450  149.8700  150.2050  180296
       2020-07-31 10:15:00  150.9200  150.9300  150.2100  150.4699  140382
       2020-07-31 10:00:00  151.0600  151.2000  150.6200  150.8800  111662
       2020-07-31 09:45:00  151.6800  151.7500  151.0600  151.1000  218477
       2020-07-31 09:30:00  151.8000  151.8000  151.4600  151.4600    1290
       2020-07-30 16:00:00  152.2450  152.3500  151.9700  152.2000  291367
       2020-07-30 15:45:00  152.3500  152.5000  152.1900  152.2500  149306
       2020-07-30 15:30:00  152.0600  152.4300  151.9500  152.3900   84332
...                              ...       ...       ...       ...     ...
ABT    2020-07-30 10:15:00  100.5500  100.9200  100.2400  100.5200  127163
       2020-07-30 10:00:00  100.6000  100.9900  100.3800  100.5900  225781
       2020-07-30 09:45:00  100.9800  101.1500  100.4100  100.6000  394068
       2020-07-30 09:30:00  102.0600  102.0600  100.9800  100.9800    4226
       2020-07-29 16:00:00  102.6800  102.7100  102.3300  102.5400  478041
       2020-07-29 15:45:00  102.7100  102.8931  102.6300  102.6800  206189
       2020-07-29 15:30:00  102.5100  102.8300  102.5000  102.7250  160667
       2020-07-29 15:15:00  102.6200  102.6400  102.3100  102.5000  143452
       2020-07-29 15:00:00  102.8200  102.8200  102.6000  102.6100  126025
       2020-07-29 14:45:00  102.6900  102.8700  102.6500  102.8248  147191
       2020-07-29 14:30:00  102.6700  102.7400  102.5500  102.6800   97410
       2020-07-29 14:15:00  102.5400  102.7700  102.3900  102.6500  116807
       2020-07-29 14:00:00  102.4700  102.5900  102.4300  102.5900  118649
       2020-07-29 13:45:00  102.3950  102.5000  102.3426  102.4700   90417
       2020-07-29 13:30:00  102.5500  102.6600  102.3850  102.3950   95013
       2020-07-29 13:15:00  102.4700  102.5500  102.4000  102.5350   84245
       2020-07-29 13:00:00  102.3750  102.4800  102.3400  102.4650   95445
       2020-07-29 12:45:00  102.2700  102.5450  102.2450  102.3700  106987
       2020-07-29 12:30:00  102.2500  102.2900  102.0900  102.2500   93153
       2020-07-29 12:15:00  102.4700  102.4843  102.1600  102.2700  125330
       2020-07-29 12:00:00  102.5500  102.5600  102.3700 

### Scratch

In [36]:
meta_data

{'1. Information': 'Intraday (15min) open, high, low, close prices and volume',
 '2. Symbol': 'GOOGL',
 '3. Last Refreshed': '2020-07-31 19:30:00',
 '4. Interval': '15min',
 '5. Output Size': 'Compact',
 '6. Time Zone': 'US/Eastern'}

In [15]:
type(s.prices_df.index.get_level_values('ts'))

pandas.core.indexes.datetimes.DatetimeIndex

In [47]:
# x = s.prices_df[s.prices_df.index.get_level_values('ticker')=='MMM']